In [1]:
from glob import glob
import codecs
from datetime import datetime
import re
import shutil
from bs4 import BeautifulSoup
import os
from time import sleep
from selenium import webdriver
import pandas as pd

In [2]:
def make_github_blog(df, file, blog_path, file_path, blog, excerpt):
    # 셀레니움 실행
    url = df[df['Name'] == file.split('\\')[-1].split('.html')[0]]['URL'].values[0]
    
    driver = webdriver.Chrome()
    driver.get(url)
    
    
    for i in glob('*.html'):
        os.remove(i)
        
    sleep(6)
    
    directory = file.split('.html')[0]
    
    # layout 생성
    file_name = file.split('\\')[-1].split('.html')[0]
    html = codecs.open(file, 'r', 'utf-8').read()
    title = BeautifulSoup(html).find_all('title')[0].get_text()
    date = html[html.find('<time>')+7:html.find('</time>')]
    date = datetime.strptime(date, '%b %d, %Y %H:%M %p')
    date = '%s-%s-%s' % (date.year, date.month, date.day)
    feature = html[html.find('<img class="page-cover-image"'):]
    feature = feature[:feature.find('" style')]
    
    
    # content 생성
    style = '<style>\n' + html[html.find('a,\na'):html.find('</style>') + 8]
    original_style = 'h1' + style.split('h1')[2].split('.source')[0]
    
    changed_style = '''
    h1 {
        font-size: 1.875rem;
        margin-top: 5.5rem; 
    }

    h2 {
        font-size: 1.5rem;
        margin-top:4rem;
    }

    h3 {
        font-size: 1.25rem;
        margin-top: 3.5rem;
    }
    '''
    
    style = style.replace(original_style, changed_style)
    content = html[html.find('''<div class="page-body">'''):html.find('</article>')]
    
    # 이미지
    folder = feature[feature.find('src'):].split('/')[0][5:]
    feature = feature[feature.find('src'):][5:].replace(folder, './assets/img/%s' % folder)
    content = content.replace(folder, '../assets/img/%s' % folder)
    
    tags = []
    start = html.find('Tag')

    if start == -1:
        tags = ''
    else:
        for i in BeautifulSoup(html[start:html.find('page-body')]).find_all('span'):
            tags.append(i.get_text())
            
    tags = str(tags).replace('\'', '')
    
    while True:
        try:
            # 수식 바꾸기
            math = [i.text.split('\n')[0] for i in driver.find_elements_by_class_name("notion-selectable.notion-equation-block")]
            res = [i for i in range(len(content)) if content.startswith('<div class="equation-container">', i)] 
            sum_ = 0
            for num, i in enumerate(res):
                i += sum_
                start = i
                end = i + len(content[i:].split('</div>')[0] + '</div>')

                add_content = '<div>'+'$$'+math[num]+'$$'+'</div>'
                content = content[:start] + add_content +content[end:]
                sum_ += len(add_content) - end + start

            driver.close()
            break
            
        except:
            if len(res) == 0:
                break
            pass
    
    blog_ = ''
    if blog:
        blog_ = '\nblog: true'

    layout = '''---
layout: post
title: "%s"
date:   %s
excerpt: "%s"%s
tags: %s
feature: .%s 
comments: true
---


    ''' % (title, date, excerpt, blog_, tags, feature)
    print(layout)
    html_name = ('%s-%s.html' % (date, title)).replace(' ', '-').replace(':', '_')
    html_name = ['_' if i in ['\\', '|', '?', '*', '>', '<', ':', '/'] else i for i in html_name]
    html_name = ''.join(html_name)
    
    with open(html_name, 'w', encoding='utf-8') as f:
        f.write('%s\n%s\n%s' % (layout, style, content))
        
        
    while True:
        print('wait')
        if os.path.isfile(html_name):
            if os.path.getsize(html_name) > 5000:
                break

            
    shutil.copytree(directory, directory+'_copy')

    
    try:
        shutil.move(directory, blog_path + '/assets/img')
        os.rename(directory+'_copy', directory)
    except:
        print('directory already exists')
        shutil.rmtree(directory+'_copy')

    try:
        shutil.move(html_name, blog_path + '/_posts')
    except:
        print('file already exists')

# 바꿀 항목!!

In [3]:
# notion web url
# url = 'https://www.notion.so/Learning-Deep-Features-for-Discriminative-Localization-eb1f4a2fef20496b8300132139a7c1c2'

# github io path
blog_path = '../sangyeop-kim.github.io'

# saved export path 
file_path = ('../../블로그/github blog/')
file = glob(file_path + '*.html')

df = pd.read_excel(file_path + 'url.xlsx')

blog = True
excerpt = ''

In [5]:
for f in file:
    make_github_blog(df, f, blog_path, file_path, blog, excerpt)

---
layout: post
title: "Fast R-CNN"
date:   2020-5-18
excerpt: ""
tags: [Object detection, image]
feature: ../assets/img/Fast%20R-CNN%20cb32e07e9980474dad1c093830809fcc/Fast_RCNN1.jpg 
comments: true
---


    
wait
---
layout: post
title: "What is the fastest way to load a big dataset?"
date:   2020-9-12
excerpt: ""
tags: 
feature: ../assets/img/What%20is%20the%20fastest%20way%20to%20load%20a%20big%20dataset%20d9e9b0322cd84b9cb5718a401aa44c71/the_fastest.jpg 
comments: true
---


    
wait
---
layout: post
title: "내가 만들어 쓰는 Analysis-template (pytorch)"
date:   2020-8-28
excerpt: ""
tags: 
feature: ../assets/img/%E1%84%82%E1%85%A2%E1%84%80%E1%85%A1%20%E1%84%86%E1%85%A1%E1%86%AB%E1%84%83%E1%85%B3%E1%86%AF%E1%84%8B%E1%85%A5%20%E1%84%8A%E1%85%B3%E1%84%82%E1%85%B3%E1%86%AB%20Analysis-template%20(pytorch)%20666b821e31f349fdb5be876c8e0ec07b/torch_wallpaper.png 
comments: true
---


    
wait
---
layout: post
title: "파이참 서버 연결하기."
date:   2020-6-6
excerpt: ""
tags: 
feature: ../assets/img/%E